# Visitor: classic visitor (refined)

> Taking advantage of dynamic typing

The classic/canonical implementation of the visitor design pattern that we saw in the previous lesson works great for statically typed languages, but Python is dynamically typed, which means that we can simplify the implementation and still get the same behavior and advantages.

First of all, let's bring in the decorator code from the previous lesson.

In [1]:
# taken from https://tavianator.com/the-visitor-pattern-in-python/

def _qualname(obj):
    """Get the fully-qualified name of an object (including module)."""
    return obj.__module__ + '.' + obj.__qualname__

def _declaring_class(obj):
    """Get the name of the class that declared an object."""
    name = _qualname(obj)
    return name[:name.rfind('.')]

# Stores the actual visitor methods
_methods = {}

# Delegating visitor implementation
def _visitor_impl(self, arg):
    """Actual visitor method implementation."""
    method = _methods[(_qualname(type(self)), type(arg))]
    return method(self, arg)

# The actual @visitor decorator
def visitor(arg_type):
    """Decorator that creates a visitor method."""
    def decorator(fn):
        declaring_class = _declaring_class(fn)
        _methods[(declaring_class, arg_type)] = fn
        # Replace all decorated methods with _visitor_impl
        return _visitor_impl
    return decorator

As for the simplification: instead of performing the **double dispatch**, we can simply omit the `accept` method from the expression classes and modify the `visit` method so that it calls itself passing the appropiate parameter, like this:

In [ ]:
class DoubleExpression:
    def __init__(self, value):
        self.value = value

class AdditionExpression:
    def __init__(self, left, right):
        self.left = left
        self.right = right

class ExpressionPrinter:
    def __init__(self):
        self.buffer = []

    @visitor(DoubleExpression)
    def visit(self, de):
        self.buffer.append(str(de.value))

    @visitor(AdditionExpression)
    def visit(self, ae):
        """We don't call accept anymore; we call visit instead"""
        self.buffer.append('(')
        self.visit(ae.left)
        self.buffer.append('+')
        self.visit(ae.right)
        self.buffer.append(')')

    def __str__(self):
        return ''.join(self.buffer)

In [6]:
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)
printer = ExpressionPrinter()
printer.visit(e)
print(printer)

(1+(2+3))


In the code above, when we call `visit`, `ExpressionPrinter` already knows which version it needs to invoke thanks to our decorators, so there is no need to go back and forth from `visit` to `accept`, because the decorator handles **dynamic dispatch** by type.

However, we're losing any kind of explicit requirement for supporting specific structures: in the previous code, should we forget to implement `accept` in one of the classes, the program would fail, but in this version there is no interface to implement explicitly which can lead to unexpected side effects such as the program seemingly running OK but with wrong results, which can be difficult to debug.

As a rule of thumb, for simpler projects, this kind of simplified implementation should be good enough, but for more complex projects, enforcing an interface that every affected class needs to be implemented can help improve the code quality and avoid errors.

We will now implement a **stateful visitor**: the `ExpressionEvaluator` class. We will store the value of a `DoubleExpression` and we will carefully keep the left side of an `AdditionExpression` on a temporal variable so that we don't overwrite it when evaluating the right side:

In [ ]:
class ExpressionEvaluator:
  def __init__(self):
    self.value = None

  @visitor(DoubleExpression)
  def visit(self, de):
    self.value = de.value

  @visitor(AdditionExpression)
  def visit(self, ae):
    self.visit(ae.left)
    temp = self.value # we temporarily store the left side value so we don't overwrite it in the next step
    self.visit(ae.right)
    self.value += temp # we add both sides

We could make a much cleaner version of this visitor by having `visit(DoubleExpression)` return the value and having `visit(AdditionExpression)` recursively call itself on both the left and right sides and return the sum; this would have the added benefit of making the visitor **stateless**, but the point of this example is that sometimes you need to be careful when dealing with stateful visitors because you need to make sure that you don't alter the state in unexpected ways.

We can now use both of our visitors at the same time, like this:

In [8]:
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)
printer = ExpressionPrinter()
printer.visit(e)

evaluator = ExpressionEvaluator()
evaluator.visit(e)

print(f'{printer} = {evaluator.value}')

(1+(2+3)) = 6
